In [1]:
import json
import pathlib
from pprint import pprint

json_filename = 'installers.json'

json_file = pathlib.Path(json_filename)

In [2]:
with open(json_file, 'r') as f:
    installers_dicts = json.load(f)

In [6]:
installer_id = '2020.02_py3.8_win-64'
pkgs_dict = {}

In [7]:
pprint(installers_dicts[installer_id]['pkgs']['affine'])

{'included': False,
 'link': 'https://github.com/sgillies/affine',
 'summary': 'Matrices describing affine transformation of the plane. / '
            'BSD-3-Clause',
 'version': '2.3.0'}


In [ ]:
for pkg in installers_dicts[installer_id]['pkgs'].keys():

    # pkgs_dict[pkg] = {}
    # pkgs_dict[pkg][]